### Todo: lay embedding hai tam hinh, tinh cosine similatiry giua embedding hai cai do

In [37]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis
from sklearn.metrics.pairwise import cosine_similarity

app = FaceAnalysis(name="buffalo_l", allowed_modules=["detection", "recognition"])
app.prepare(ctx_id=0, det_size=(640, 640))

img0 = cv2.imread('backend/img/images/b629a8b5-f154-4db2-b6d8-142e3b5843b2_kendall.jpeg')
img1 = cv2.imread('backend/img/profile/84543041-0d01-44cb-8195-054043dd105a_kendall-jenner.png')

def get_embed(img):
    faces = app.get(img)
    if img is None:
        print("Could not open or find the image")
    print(faces)
    for face in faces:
        return face['embedding']

img0 = get_embed(img0).reshape(1,-1)
img1 = get_embed(img1).reshape(1,-1)

def identical(i1,i2):
    i1 = i1.flatten()
    i2 = i2.flatten()
    cosine = (i1 @ i2) / (np.linalg.norm(i1) * np.linalg.norm(i2))
    print(cosine)
    if cosine >= 0.65:
        print('BTien')
    else:
        print('Not BTien')

cosine_similarity(img0, img1)

/opt/homebrew/Caskroom/miniforge/base/envs/reconnect/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/jennifertran/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/jennifertran/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/jennifertran/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /Users/jennifertran/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /Users/jennifertran/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)
[{'bbox': array([385.673

array([[0.65231824]], dtype=float32)

In [43]:
from sklearn.metrics.pairwise import cosine_similarity

upload_embeds = np.load('backend/uploads.npy') # embed của toàn bộ ảnh upload
user_embeds = np.load('backend/users.npy') # embed của toàn bộ ảnh user 

user_embeds.shape, upload_embeds.shape

a = cosine_similarity(upload_embeds, user_embeds)
print(a)
user_match_pair_ids = np.argmax(a, axis=1)
a[user_match_pair_ids]

[[0.08730046 0.65231824]
 [0.67154235 0.08976456]]


array([[0.67154235, 0.08976456],
       [0.08730046, 0.65231824]], dtype=float32)

In [45]:
import pickle

with open('backend/uploads.pkl', 'rb') as f:
    pair_mapping_ids = pickle.load(f)
with open('backend/users.pkl', 'rb') as f:
    user_mapping_ids = pickle.load(f)

pair_mapping_ids, user_mapping_ids

([2, 4], [1, 3])

In [59]:
upload_embeds = np.load('backend/uploads.npy') # embed của toàn bộ ảnh upload
user_embeds = np.load('backend/users.npy') # embed của toàn bộ ảnh user 

pair_indices = [0]
user_indices = [0, 1]
pair_ids = [2]
user_ids = [3]

pair_embeds = upload_embeds[pair_indices]
valid_user_embeds = user_embeds[user_indices]

matrix = cosine_similarity(pair_embeds, valid_user_embeds)
print("matrix", matrix.shape)
# Được một cái matrix - dòng là pair_embeds còn cột là valid_user_embeds

# argmax lấy ra id của user giống với pair nhất, axis = 1
user_match_pair_ids = np.argmax(matrix, axis=1)
print("Hellooooooooo user_match_pair_ids", user_match_pair_ids)

# tìm value của user giống pair nhất
user_match_pair = matrix[:,user_match_pair_ids]
print("Helloooooooo user_match_pair", user_match_pair)

# so sánh với threshold
threshold = 0.62
thr_filter = user_match_pair > threshold
print("thr_filter", thr_filter)

# tìm match_score
match_score = user_match_pair[thr_filter].tolist()
print("Helloooooooo match_score", match_score)


# lấy pair_ids hợp lệ và user_ids hợp lệ
print("Helloooooooo pair_ids", pair_ids)
print("Helloooooooo user_ids", user_ids)


valid_pair_ids = np.array(pair_ids)[thr_filter[0]].tolist()
valid_user_ids = np.array(user_ids)[thr_filter[0]].tolist()

tuple_valid_pair_ids = tuple(valid_pair_ids)
tuple_valid_user_ids = tuple(valid_user_ids)

print('valid_pair_ids', valid_pair_ids)
print('valid_user_ids', valid_user_ids)

if len(valid_pair_ids) == 1:
    tuple_valid_pair_ids = int(valid_pair_ids[0])
if len(valid_user_ids) == 1:
    tuple_valid_user_ids = int(valid_user_ids[0])

print("Helloooooooo tuple_valid_pair_ids", tuple_valid_pair_ids)
print("Helloooooooo tuple_valid_user_ids", tuple_valid_user_ids)


matrix (1, 2)
Hellooooooooo user_match_pair_ids [1]
Helloooooooo user_match_pair [[0.64284265]]
thr_filter [[ True]]
Helloooooooo match_score [0.6428426504135132]
Helloooooooo pair_ids [2]
Helloooooooo user_ids [3]
valid_pair_ids [2]
valid_user_ids [3]
Helloooooooo tuple_valid_pair_ids 2
Helloooooooo tuple_valid_user_ids 3


In [63]:
user_match_pair_ids = np.array([1, 0])
matrix = np.array([[0.09776807, 0.65231824],
                   [1.0000001,  0.060811  ]])
user_match_pair = matrix[:,user_match_pair_ids]
print("Helloooooooo user_match_pair", user_match_pair)

Helloooooooo user_match_pair [[0.65231824 0.09776807]
 [0.060811   1.0000001 ]]


In [85]:
pair_ids = [3, 8]
user_ids = [4, 6]
matrix = np.array([[0.09776807, 0.65231824],
                   [1.0000001,  0.060811  ]])

threshold = 0.62
valid_pair_user_ids = []
for i in range(len(pair_ids)):
    for j in range(len(user_ids)):
            if matrix[i][j] >= threshold:
                valid_pair_user_ids.append((pair_ids[i], user_ids[j]))
valid_pair_user_ids = np.array(valid_pair_user_ids)

print("valid_pair_user_ids", valid_pair_user_ids)

# tìm value của user giống pair nhất
user_match_pair_ids = np.argmax(matrix, axis=1)
user_match_pair = matrix[np.arange(matrix.shape[0]), user_match_pair_ids]
thr_filter = user_match_pair > threshold

print("Hellooooooooo user_match_pair_ids", user_match_pair_ids)
print("Helloooooooo user_match_pair", user_match_pair)
print("thr_filter", thr_filter)

# tìm match_score
match_score = user_match_pair[thr_filter].tolist()
print(match_score)

valid_pair_user_ids[:,1]


valid_pair_user_ids [[3 6]
 [8 4]]
Hellooooooooo user_match_pair_ids [1 0]
Helloooooooo user_match_pair [0.65231824 1.0000001 ]
thr_filter [ True  True]
[0.65231824, 1.0000001]


array([6, 4])

### Todo: lay embed btienne so sanh voi cai frame vong lap. Giong nhau thi ve ten Bao Tien len tren toa do bbox

In [ ]:
import cv2
import numpy as np
from insightface.app import FaceAnalysis

window_name = 'Image'
app = FaceAnalysis(name="buffalo_sc", providers=['CPUExecutionProvider'])
app.prepare(ctx_id=0, det_size=(640, 640))


def get_embed(img):
    faces = app.get(img)
    if img is None:
        print("Could not open or find the image")
    for face in faces:
        return face['embedding'].astype(int)
    
def identical(i1,i2):
    i1 = i1.flatten()
    i2 = i2.flatten().T
    cosine = (i1 @ i2) / (np.linalg.norm(i1) * np.linalg.norm(i2))
    if cosine > 0.3:
        print('BTien')
    else:
        print('Not BTien')


cap = cv2.VideoCapture(0) # mở webcam

img1 = cv2.imread('static/img/btienne.jpg')
btienne = get_embed(img1)
# Check if the webcam is opened correctly
if not cap.isOpened():
    raise IOError("Cannot open webcam")

while True:
    ret, img = cap.read() # ret: take frame successfully or not
    if not ret:
        break

    faces = app.get(img)
    for face in faces:
        bbox = face['bbox'].astype(int)
        start_point = bbox[:2]
        end_point = bbox[2:]
        thickness = 2
        img = cv2.rectangle(img, start_point, end_point, (255, 0, 0) , thickness)

        keypoints = face['kps'].astype(int)
        for point in keypoints:
            img = cv2.circle(img, point, 3, (0, 0, 255) , thickness)

    on_vid = get_embed(img)
    identical(btienne, on_vid)
    
    cv2.imshow(window_name, img) 

    c = cv2.waitKey(1) # wait 1ms user press button
    if c == 27: # ESC button
        break


cap.release()
cv2.waitKey(0)
cv2.destroyAllWindows()

# bbox là khung bao quanh đầu
# kps là bao gồm toạ độ mắt mũi miệng
# det_score là phần trăm có gương mặt
# embeddings là array nhận diện gương mặt

In [13]:
import numpy as np
matrix = np.random.normal(size=(20,10))
ids = np.argmax(matrix, axis=1)
ids.shape

(20,)

In [15]:
pair_ids = [1,5,7]
pair_mapping_ids = [3, 1, 6, 8, 5, 7]

indices = [pair_mapping_ids.index(id) for id in pair_ids]
matrix[indices]

ValueError: [1, 5, 7] is not in list

In [16]:
a = np.array([2,3])
tuple(a)

(2, 3)

In [29]:
a = (5,)
if len(a) == 1:
    print(100)

100
